In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import imageio
import cv2
from scipy import interpolate

In [3]:
def quick_rundown(bog,w,width,fig):
    (x,y,chs) = bog.shape
    num_plots = 9
    indices = ((i*width)+w+1 for i in range(num_plots))
    subplots = (fig.add_subplot(num_plots,width,i) for i in indices)
    # The image itself
    original = subplots.__next__()
    original.set_title('original')
    original.imshow(bog.astype('double')/65535.0)
    # channel intensity distributions
    distribution = subplots.__next__()
    distribution.set_title('per-channel intensity distribution')
    logdist = subplots.__next__()
    logdist.set_yscale('log')
    logdist.set_title('per-channel intensity distribution, log')
    noisedist = subplots.__next__()
    noisedist.set_yscale('log')
    noisedist.set_title('per-channel intensity distribution, log, 1000-12000')
    lodist = subplots.__next__()
    lodist.set_yscale('log')
    lodist.set_title('per-channel intensity distribution, log, 0-1000')
    hidist = subplots.__next__()
    hidist.set_yscale('log')
    hidist.set_title('per-channel intensity distribution, log, 64000-65535')
    for (i,color) in enumerate("rgb"[:chs]):
        counts = np.bincount(bog[:,:,i].reshape(x*y))
        distribution.plot(counts,color)
        logdist.plot(counts,color)
        noisedist.plot(counts[1000:12000],color)
        lodist.plot(counts[:1000],color)
        hidist.plot(counts[64000:],color)
    # Pick out where the bulk of the pixels fall
    loband = subplots.__next__()
    loband.set_title('low intensity pixels shown in white')
    loband.imshow(((bog <= 1000)).astype('float'))
    noiseband = subplots.__next__()
    noiseband.set_title('medium-low intensity pixels shown in white')
    noiseband.imshow(((bog > 1000) & (bog < 12000)).astype('float'))
    hiband = subplots.__next__()
    hiband.set_title('high intensity pixels shown in white')
    hiband.imshow(((bog >= 12000)).astype('float'))
def quick_rundowns(bogs):
    width = len(bogs)
    fig = plt.figure(figsize=(15,40))
    for (w,img) in enumerate(bogs):
        quick_rundown(img,w,width,fig)
    return fig

In [4]:
# fig = quick_rundowns([t1,t2])

In [5]:
# fig.savefig('fig.pdf',dpi=600)

In [6]:
def match(im1,im2):
    im1 = (im1/256).astype('uint8')
    im2 = (im2/256).astype('uint8')
    det = cv2.ORB_create(nfeatures=1000)
    kp1,desc1 = det.detectAndCompute(im1,None)
    kp2,desc2 = det.detectAndCompute(im2,None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(desc1,desc2)
    matches = sorted(matches, key = lambda x:x.distance)
    matches = matches[:len(matches)//10]
    assert(len(matches)>10)
    src_pts = np.float32([ kp1[m.queryIdx].pt for m in matches ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m.trainIdx].pt for m in matches ]).reshape(-1,1,2)
    # Mask tells us which matches seem internally consistent
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
    return M
#     matches = cv2.drawMatches(im1,kp1,im2,kp2,[matches[i] for i in range(len(matches)) if mask[i]],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
#     imageio.imsave('matches.png',matches)

def add(im,ims):
    (h,w,_) = im.shape
    out = im.astype('double')
    count = np.full((h,w),1.0)
    for load_im2 in ims:
        im2 = load_im2(None)
        M = match(im2,im)
        source = im2.astype('double')
        out += cv2.warpPerspective(source,M,(w,h))
        counter = np.full(source.shape[0:2],1.0)
        count += cv2.warpPerspective(counter,M,(w,h))
    return (out/out.max(),count/count.max())
        
# M = match(t2,t1)

# (h,w,_) = t1.shape
# x = cv2.warpPerspective(t2.astype('double'),M,(w,h))


In [8]:
combined,count = add(imageio.imread('3200-10-2.8.tiff'),[(lambda _ : imageio.imread(path)) for path in ['6400-5-2.8.tiff']])
valid_region = count > 0.99
for ch in [0,1,2]:
    combined[:,:,ch] *= valid_region
    combined[:,:,ch] += -(valid_region - 1)
plt.imsave('combined__.tiff',combined)

In [ ]:
# fig = quick_rundowns([t1,(t3*65535).astype('uint16')])
# fig.savefig('combined.pdf',dpi=600)



In [ ]:
x = np.array([1]).astype('uint8').dtype


In [ ]:
{x : 4}[x]

In [ ]:
np.uint8